Descompresion de datos

In [2]:
import zipfile
import os

def descomprimir_zips_en_datos():
    """
    Descomprime todos los archivos ZIP encontrados en el directorio actual
    dentro de una subcarpeta llamada 'datos'.
    """
    carpeta_destino = "datos"

    # Crear la carpeta 'datos' si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
        print(f"Carpeta '{carpeta_destino}' creada.")

    # Obtener una lista de todos los archivos en el directorio actual
    archivos_en_directorio = os.listdir('.')

    # Filtrar solo los archivos .zip
    archivos_zip = [f for f in archivos_en_directorio if f.endswith('.zip')]

    if not archivos_zip:
        print("No se encontraron archivos ZIP en el directorio actual.")
        return

    print(f"Archivos ZIP encontrados: {archivos_zip}")

    for archivo_zip in archivos_zip:
        ruta_completa_zip = os.path.join('.', archivo_zip)
        try:
            with zipfile.ZipFile(ruta_completa_zip, 'r') as zip_ref:
                zip_ref.extractall(os.path.join('.', carpeta_destino))
            print(f"'{archivo_zip}' descomprimido exitosamente en '{carpeta_destino}'.")
        except zipfile.BadZipFile:
            print(f"Error: '{archivo_zip}' no es un archivo ZIP válido o está corrupto.")
        except Exception as e:
            print(f"Ocurrió un error al descomprimir '{archivo_zip}': {e}")

descomprimir_zips_en_datos()

Archivos ZIP encontrados: ['estadisticas_normales_9120.zip', 'Registro_temperaturas-05072025.zip']
'estadisticas_normales_9120.zip' descomprimido exitosamente en 'datos'.
'Registro_temperaturas-05072025.zip' descomprimido exitosamente en 'datos'.


In [3]:
import pandas as pd
import os
import numpy as np


def cargar_excel_desde_datos(nombre_archivo):
    """
    Carga un archivo Excel desde la subcarpeta 'datos'.

    Args:
        nombre_archivo (str): El nombre completo del archivo Excel a cargar.

    Returns:
        pd.DataFrame: El DataFrame de pandas con los datos del Excel, o None si hay un error.
    """
    ruta_completa = os.path.join('datos', nombre_archivo)

    if not os.path.exists(ruta_completa):
        print(f"Error: El archivo '{ruta_completa}' no se encontró.")
        return None

    try:
        df = pd.read_excel(ruta_completa, header=4)
        print(f"Archivo '{nombre_archivo}' cargado exitosamente desde 'datos/'.")
        print("Primeras 5 filas del DataFrame:")
        print(df.head())
        return df
    except Exception as e:
        print(f"Ocurrió un error al cargar el archivo Excel: {e}")
        return None
    
nombre_excel = "Estadísticas normales Datos abiertos 1991-2020.xlsx"
df = cargar_excel_desde_datos(nombre_excel)

if df is not None:
    print("\nInformación del DataFrame:")
    df.info()
    df.head()


Archivo 'Estadísticas normales Datos abiertos 1991-2020.xlsx' cargado exitosamente desde 'datos/'.
Primeras 5 filas del DataFrame:
                 Estación                           Valor Medio de   Ene  \
0  LA QUIACA OBSERVATORIO                         Temperatura (°C)  13.2   
1  LA QUIACA OBSERVATORIO                  Temperatura máxima (°C)  20.6   
2  LA QUIACA OBSERVATORIO                  Temperatura mínima (°C)   7.7   
3  LA QUIACA OBSERVATORIO                     Humedad relativa (%)  62.6   
4  LA QUIACA OBSERVATORIO  Velocidad del Viento (km/h) (2011-2020)   6.5   

    Feb   Mar   Abr   May   Jun   Jul   Ago   Sep   Oct   Nov   Dic  
0    13  12.8  11.3   7.3   4.8   4.5     7    10  12.4  13.4  13.9  
1  20.4  20.6  20.3  17.8  16.3  16.1    18    20  21.7  22.5  22.2  
2   7.6   6.6   3.1  -2.5  -5.7  -6.2    -4  -0.4   3.3   5.5   7.3  
3  63.2  60.3    46  32.6  27.4  25.7  26.7  32.1  42.4  48.6  55.8  
4   6.8   6.7   5.5   4.8   5.5   5.9   6.7   7.9   7.9   7.7 

In [5]:
df.describe()
df.columns

# 1. Renombrar columnas para mayor claridad
nuevos_nombres = {
    'Valor Medio de': 'Tipo_Variable',
    'Temperatura (°C)': 'Temperatura_C',
    'Temperatura máxima (°C)': 'Temperatura_Maxima_C',
    'Temperatura mínima (°C)': 'Temperatura_Minima_C',
    'Humedad relativa (%)': 'Humedad_Relativa_Porcentaje',
    'Velocidad del Viento (km/h) (2011-2020)': 'Velocidad_Viento_kmh',
    'Nubosidad total (octavos)': 'Nubosidad_Octavos',
    'Precipitación (mm)': 'Precipitacion_mm',
    'Frecuencia de días con Precipitación superior a 1.0 mm': 'Frecuencia_Dias_Precipitacion_gt_1mm'
}
df.rename(columns={k: v for k, v in nuevos_nombres.items() if k in df.columns}, inplace=True)
print("\n--- Columnas renombradas ---")
print(df.columns.tolist())

# 2. Manejo de valores faltantes (S/D) y conversión a numérico
meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

# Combinar las columnas que deberían ser numéricas
columnas_a_procesar = list(meses) + [
    'Temperatura_C', 'Temperatura_Maxima_C', 'Temperatura_Minima_C',
    'Humedad_Relativa_Porcentaje', 'Velocidad_Viento_kmh',
    'Nubosidad_Octavos', 'Precipitacion_mm', 'Frecuencia_Dias_Precipitacion_gt_1mm'
]
# Filtrar solo las columnas que existen en el DataFrame
columnas_a_procesar = [col for col in columnas_a_procesar if col in df.columns]

for col in columnas_a_procesar:
    # Opción 1: Reemplazar y luego convertir. La advertencia aparece en el replace.
    # Para evitarla aquí, podemos asegurar que el dtype sea objeto antes de reemplazar si no lo es.
    # Sin embargo, el replace por sí mismo es el que genera la advertencia si intenta downcast.
    # Una forma explícita es:
    df[col] = df[col].astype(str).replace('S/D', np.nan) # Asegurar que es str antes de replace para evitar downcasting inesperado
    df[col] = pd.to_numeric(df[col], errors='coerce')


# 3. Limpieza de la columna 'Estación' (manejo del asterisco)
df['Estacion_Limpia'] = df['Estación'].astype(str).str.replace('*', '', regex=False).str.strip()
df['Periodo_Normalizado_Completo'] = ~df['Estación'].astype(str).str.contains('*', regex=False)


# 4. Eliminar filas completamente vacías al final
df.dropna(how='all', subset=['Estación', 'Tipo_Variable'] + meses, inplace=True)


print("\n--- DataFrame después de preprocesamiento (primeras 10 filas) ---")
print(df.head(10))
print("\n--- Información del DataFrame después de preprocesamiento ---")
df.info()
print("\n--- Valores nulos por columna después de preprocesamiento ---")
print(df.isnull().sum())


--- Columnas renombradas ---
['Estación', 'Tipo_Variable', 'Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic', 'Estacion_Limpia', 'Periodo_Menor']

--- DataFrame después de preprocesamiento (primeras 10 filas) ---
                 Estación                                      Tipo_Variable  \
0  LA QUIACA OBSERVATORIO                                   Temperatura (°C)   
1  LA QUIACA OBSERVATORIO                            Temperatura máxima (°C)   
2  LA QUIACA OBSERVATORIO                            Temperatura mínima (°C)   
3  LA QUIACA OBSERVATORIO                               Humedad relativa (%)   
4  LA QUIACA OBSERVATORIO            Velocidad del Viento (km/h) (2011-2020)   
5  LA QUIACA OBSERVATORIO                          Nubosidad total (octavos)   
6  LA QUIACA OBSERVATORIO                                 Precipitación (mm)   
7  LA QUIACA OBSERVATORIO  Frecuencia de días con Precipitación superior ...   
8              ORÁN AERO*        

In [ ]:
import pandas as pd
import numpy as np
import os

# Opcional: Para mejorar la visualización en el notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

nombre_excel = "Estadísticas normales Datos abiertos 1991-2020.xlsx"
ruta_completa = os.path.join('datos', nombre_excel)

if not os.path.exists(ruta_completa):
    print(f"Error: El archivo '{ruta_completa}' no se encontró. Asegúrate de que el archivo ZIP se descomprimió correctamente y el archivo .xlsx está en la carpeta 'datos'.")
else:
    print(f"Intentando cargar '{nombre_excel}' sin especificar 'header' inicialmente para inspeccionar...")
    try:
        # Cargar sin especificar header para ver el raw data y encontrar la fila correcta
        df_raw = pd.read_excel(ruta_completa, header=None)

        print("\n--- Primeras 10 filas del archivo SIN HEADER ESPECIFICADO ---")
        # .to_string() es útil para ver todas las columnas sin truncar en el notebook
        print(df_raw.head(10).to_string())

        print("\n--- Tipo de datos de las columnas SIN HEADER ESPECIFICADO ---")
        df_raw.info()

        # Intentar encontrar la fila que contiene "Estación" y "Valor Medio de"
        # Buscamos en las primeras 10 filas para ser eficientes
        header_row_index = -1
        for i in range(min(10, len(df_raw))):
            # Convertir la fila a string y a minúsculas para una búsqueda insensible a mayúsculas y manejar NaNs
            row_as_str = df_raw.iloc[i].astype(str).str.lower()
            if ('estación' in row_as_str.values and 'valor medio de' in row_as_str.values):
                header_row_index = i
                break

        if header_row_index != -1:
            print(f"\n¡Encabezado detectado en la fila (0-indexada): {header_row_index}!")
            print(f"Las columnas en esa fila son: {df_raw.iloc[header_row_index].tolist()}")
        else:
            print("\nNo se pudo encontrar la fila del encabezado que contiene 'Estación' y 'Valor Medio de' en las primeras 10 filas.")
            print("Por favor, revise la salida anterior para identificar la fila manualmente.")
            print("Si la fila 3 (índice 3) parece correcta, establezca `header_row_index = 3` manualmente y re-ejecute la siguiente celda.")

    except Exception as e:
        print(f"Ocurrió un error al intentar la carga inicial para depuración: {e}")


Intentando cargar 'Estadísticas normales Datos abiertos 1991-2020.xlsx' sin especificar 'header' inicialmente para inspeccionar...

--- Primeras 10 filas del archivo SIN HEADER ESPECIFICADO ---
                                                                                                                                                                                                                                                                                                                                             0                                        1     2     3     4     5     6     7     8     9     10    11    12    13
0                                                                                                                                                                                                                                                                                      Estadísticas Climatológicas Normales - período 1991-2020                      

In [9]:
if 'header_row_index' not in locals() or header_row_index == -1:
    print("El índice del encabezado no fue detectado automáticamente. Por favor, revísalo en la celda anterior y establécelo manualmente.")
    print("Por ejemplo: header_row_index = 3")
else:
    try:
        # Cargar el Excel usando el header_row_index identificado
        df = pd.read_excel(ruta_completa, header=header_row_index)
        print(f"Archivo '{nombre_excel}' cargado exitosamente con encabezado en la fila {header_row_index}.")

        print("\n--- Primeras 5 filas del DataFrame después de la carga inicial ---")
        print(df.head())
        print("\n--- Nombres de columnas después de la carga inicial ---")
        print(df.columns.tolist())
        print("\n--- Información del DataFrame después de la carga inicial ---")
        df.info()

        # --- Preprocesamiento inicial ---

        # 1. Renombrar columnas para mayor claridad
        nuevos_nombres = {
            'Valor Medio de': 'Tipo_Variable',
            'Temperatura (°C)': 'Temperatura_C',
            'Temperatura máxima (°C)': 'Temperatura_Maxima_C',
            'Temperatura mínima (°C)': 'Temperatura_Minima_C',
            'Humedad relativa (%)': 'Humedad_Relativa_Porcentaje',
            'Velocidad del Viento (km/h) (2011-2020)': 'Velocidad_Viento_kmh',
            'Nubosidad total (octavos)': 'Nubosidad_Octavos',
            'Precipitación (mm)': 'Precipitacion_mm',
            'Frecuencia de días con Precipitación superior a 1.0 mm': 'Frecuencia_Dias_Precipitacion_gt_1mm'
        }
        # Solo renombra si la columna existe en el DataFrame
        df.rename(columns={k: v for k, v in nuevos_nombres.items() if k in df.columns}, inplace=True)
        print("\n--- Columnas renombradas ---")
        print(df.columns.tolist())

        # 2. Manejo de valores faltantes (S/D) y conversión a numérico
        meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

        columnas_numericas_potenciales = list(meses) + [
            'Temperatura_C', 'Temperatura_Maxima_C', 'Temperatura_Minima_C',
            'Humedad_Relativa_Porcentaje', 'Velocidad_Viento_kmh',
            'Nubosidad_Octavos', 'Precipitacion_mm', 'Frecuencia_Dias_Precipitacion_gt_1mm'
        ]
        columnas_a_procesar = [col for col in columnas_numericas_potenciales if col in df.columns]

        for col in columnas_a_procesar:
            # Reemplazar 'S/D' por NaN y luego convertir a numérico.
            df[col] = df[col].astype(str).replace('S/D', np.nan)
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # 3. Limpieza de la columna 'Estación' (manejo del asterisco)
        df['Estacion_Limpia'] = df['Estación'].astype(str).str.replace('*', '', regex=False).str.strip()
        df['Periodo_Normalizado_Completo'] = ~df['Estación'].astype(str).str.contains('*', regex=False)

        # 4. Eliminar filas completamente vacías al final
        # Usar las columnas clave para determinar si una fila está vacía
        df.dropna(how='all', subset=['Estación', 'Tipo_Variable'] + [m for m in meses if m in df.columns], inplace=True)

        print("\n--- DataFrame después de preprocesamiento inicial (primeras 10 filas) ---")
        print(df.head(10).to_string()) # Usar to_string para ver todo
        print("\n--- Información del DataFrame después de preprocesamiento inicial ---")
        df.info()
        print("\n--- Valores nulos por columna después de preprocesamiento inicial ---")
        print(df.isnull().sum())

    except Exception as e:
        print(f"Ocurrió un error durante el preprocesamiento inicial: {e}")

Archivo 'Estadísticas normales Datos abiertos 1991-2020.xlsx' cargado exitosamente con encabezado en la fila 4.

--- Primeras 5 filas del DataFrame después de la carga inicial ---
                 Estación                           Valor Medio de   Ene   Feb   Mar   Abr   May   Jun   Jul   Ago   Sep   Oct   Nov   Dic
0  LA QUIACA OBSERVATORIO                         Temperatura (°C)  13.2    13  12.8  11.3   7.3   4.8   4.5     7    10  12.4  13.4  13.9
1  LA QUIACA OBSERVATORIO                  Temperatura máxima (°C)  20.6  20.4  20.6  20.3  17.8  16.3  16.1    18    20  21.7  22.5  22.2
2  LA QUIACA OBSERVATORIO                  Temperatura mínima (°C)   7.7   7.6   6.6   3.1  -2.5  -5.7  -6.2    -4  -0.4   3.3   5.5   7.3
3  LA QUIACA OBSERVATORIO                     Humedad relativa (%)  62.6  63.2  60.3    46  32.6  27.4  25.7  26.7  32.1  42.4  48.6  55.8
4  LA QUIACA OBSERVATORIO  Velocidad del Viento (km/h) (2011-2020)   6.5   6.8   6.7   5.5   4.8   5.5   5.9   6.7   7.9   7.

In [10]:
# Asegurarse de que df exista y haya sido preprocesado
if 'df' in locals():
    # --- Reestructuración a formato "long" (tidy data) ---

    meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

    # Identificar las columnas a "derretir" (los meses)
    # Asegúrate de que 'Tipo_Variable' exista, ya que es fundamental para el melting
    id_vars = ['Estación', 'Tipo_Variable', 'Estacion_Limpia', 'Periodo_Normalizado_Completo']
    # Filtrar solo las columnas de meses que realmente existen en df.columns
    value_vars = [m for m in meses if m in df.columns]

    if not value_vars:
        print("Advertencia: No se encontraron columnas de meses para reestructurar. Verifique los nombres de las columnas en el DataFrame 'df'.")
        df_melted = None # Asegurar que df_melted no se cree si no hay meses
    else:
        try:
            df_melted = df.melt(id_vars=id_vars,
                                value_vars=value_vars,
                                var_name='Mes',
                                value_name='Valor_Mensual')

            print("\n--- DataFrame reestructurado en formato 'long' (melted) ---")
            print(df_melted.head(20).to_string()) # Mostrar más filas para ver la nueva estructura
            print("\nInformación del DataFrame melted:")
            df_melted.info()
            print("\nValores nulos en el DataFrame melted:")
            print(df_melted.isnull().sum())

        except Exception as e:
            print(f"Ocurrió un error al reestructurar el DataFrame: {e}")
            df_melted = None
else:
    print("El DataFrame 'df' no se creó en la celda anterior. No se puede reestructurar.")
    df_melted = None # Asegurar que df_melted no se defina


--- DataFrame reestructurado en formato 'long' (melted) ---
                  Estación                                           Tipo_Variable         Estacion_Limpia  Periodo_Normalizado_Completo  Mes  Valor_Mensual
0   LA QUIACA OBSERVATORIO                                        Temperatura (°C)  LA QUIACA OBSERVATORIO                          True  Ene           13.2
1   LA QUIACA OBSERVATORIO                                 Temperatura máxima (°C)  LA QUIACA OBSERVATORIO                          True  Ene           20.6
2   LA QUIACA OBSERVATORIO                                 Temperatura mínima (°C)  LA QUIACA OBSERVATORIO                          True  Ene            7.7
3   LA QUIACA OBSERVATORIO                                    Humedad relativa (%)  LA QUIACA OBSERVATORIO                          True  Ene           62.6
4   LA QUIACA OBSERVATORIO                 Velocidad del Viento (km/h) (2011-2020)  LA QUIACA OBSERVATORIO                          True  Ene            6

In [14]:
if 'df_melted' in locals() and df_melted is not None:
    # Asegurar el orden de los meses para el pivot final
    orden_meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
    if 'Mes' in df_melted.columns:
        df_melted['Mes'] = pd.Categorical(df_melted['Mes'], categories=orden_meses, ordered=True)
    else:
        print("La columna 'Mes' no se encuentra en df_melted. No se puede ordenar por meses.")

    try:
        # Primero, asegúrate de que los datos estén ordenados por estación, tipo de variable y mes
        df_sorted = df_melted.sort_values(by=['Estacion_Limpia', 'Tipo_Variable', 'Mes'])

        # Agrupar por estación y tipo de variable, y recolectar los 12 valores mensuales en una lista.
        # RENOMBRAMOS explícitamente la columna resultante a 'Vector_Mensual_Valores'.
        df_vectorized = df_sorted.groupby(['Estacion_Limpia', 'Tipo_Variable'])['Valor_Mensual'].apply(list).reset_index(name='Vector_Mensual_Valores')
        
        # print("--- df_vectorized después de agrupar y vectorizar ---")
        # print(df_vectorized.head().to_string())
        # print(df_vectorized.info())


        # Ahora pivotar este df_vectorized para que 'Tipo_Variable' se convierta en columnas
        # y los valores sean las listas de 12 meses.
        df_final_vector_pivot = df_vectorized.pivot_table(
            index='Estacion_Limpia',
            columns='Tipo_Variable',
            values='Vector_Mensual_Valores', # Ahora esta columna sí existe
            aggfunc='first'
        ).reset_index()

        # Opcional: fusionar Periodo_Normalizado_Completo si es necesario, ya que se perdió en el pivot
        df_periodo = df_melted[['Estacion_Limpia', 'Periodo_Normalizado_Completo']].drop_duplicates()
        df_final_vector_pivot = df_final_vector_pivot.merge(df_periodo, on='Estacion_Limpia', how='left')


        print("\n--- DataFrame Final con Variables como Columnas y Vectores Mensuales ---")
        # Ajusta display.max_colwidth para ver el contenido de las listas si son muy largas
        pd.set_option('display.max_colwidth', None)
        print(df_final_vector_pivot.head().to_string())
        pd.set_option('display.max_colwidth', 50) # Restaurar a un valor más manejable

        print("\nInformación del DataFrame Final:")
        df_final_vector_pivot.info()
        print("\nValores nulos en el DataFrame Final (se refieren a si la lista completa es nula):")
        print(df_final_vector_pivot.isnull().sum())

    except Exception as e:
        print(f"Ocurrió un error al realizar el pivotaje final para obtener vectores: {e}")
else:
    print("El DataFrame 'df_melted' no está disponible para el pivotaje final.")


--- DataFrame Final con Variables como Columnas y Vectores Mensuales ---
     Estacion_Limpia            Frecuencia de días con Precipitación superior a 1.0 mm                                                      Humedad relativa (%)                                     Nubosidad total (octavos)                                                               Precipitación (mm)                                                                  Temperatura (°C)                                                        Temperatura máxima (°C)                                                             Temperatura mínima (°C)                                   Velocidad del Viento (km/h) (2011-2020)  Periodo_Normalizado_Completo
0    AEROPARQUE AERO      [6.7, 6.0, 5.9, 6.6, 5.0, 4.5, 5.0, 5.0, 5.5, 7.5, 6.8, 6.6]  [67.0, 69.8, 71.3, 73.6, 76.4, 76.3, 75.5, 73.6, 71.1, 70.6, 67.0, 65.1]  [3.1, 3.2, 3.1, 3.5, 4.0, 4.2, 4.1, 3.9, 3.8, 3.9, 3.5, 3.2]  [117.5, 112.3, 111.8, 108.3, 83.3, 54.6, 64.3, 61